In [1]:
import numpy as np
import pandas as pd
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

file_path = r"C:\Users\jites\Desktop\Project_folder\Jupyter_practice\FInal_data_20_columns.xlsx"
df = pd.read_excel(file_path)

df = df.astype(str)
df.head(2)

,ChiefComplaint,PresentProblemLocation,PreProbDuration,PreProbOnset,PreProbSeverity,AttackType,SmokingHabitType,PhysicalActivity,ALCHOLIC,DietryHabit,...,TreatmentType,SurgeryType,SurgeryDurationH,SurgeryDurationM,AnesthicTypeName,AnesRecoverySmoothType,RecorveryType,Followupdate,DoctorRemark,Paragraph
0,Ho intermittent pain abdomen and flatulence X ...,Abdomen problem,Complains from 1 month ago,Gradually it happen,nan,Intermittent attack,Patient does not have smoking habit,Light Worker,Does not drink alcohol,nan,...,Treatment type OPERATIVE,Surgery type CHOLECYSTECTOMY,nan,40 minutes,Anesthic type General,Anes smooth recovery,NORMAL recovery,Follow-up date 20 December 2014,nan,Ho intermittent pain abdomen and flatulence X ...
1,During routine health checkup gallstone detect...,Abdomen problem,Complains from 2 months ago,nan,Mild problem,No attack,Patient does not have smoking habit,Moderate Worker,Drinks alcohol,Non Vegetarian,...,Treatment type OPERATIVE,Surgery type CHOLECYSTECTOMY,nan,30 minutes,Anesthic type General,Anes smooth recovery,NORMAL recovery,Follow-up date 20 December 2014,nan,During routine health checkup gallstone detect...


In [2]:
df = df.iloc[:1000]

In [3]:
def lower_text(text):
    return text.lower()

df = df.applymap(lambda x: lower_text(x))

C:\Users\jites\AppData\Local\Temp\ipykernel_21184\804839335.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: lower_text(x))


In [4]:
text_data = df['Paragraph']
target_data = df.drop(columns=['Paragraph'])

In [5]:
max_words_in_sentense_paragraph = max([len(sen.split()) for sen in text_data])
print("max_words_in_sentense_paragraph => ", max_words_in_sentense_paragraph)

max_words_in_sentense_paragraph =>  348


In [6]:
print(len(set(text_data)))

998


In [7]:
from typing import List, Dict, Tuple, Literal

In [8]:
from typing import List, Dict, Tuple, Literal


# class CustomTokenizer:
#     def __init__(self, sentences : List[str]):
#         self.sentence = sentences
#         self.index_words = dict()
#         self.word_index = dict()
#         self.unique_words = set()

#     def find_unique_words(self):
#         all_sentences_words = []
#         for sentence in self.sentence:
#             words_list = sentence.split()
#             all_sentences_words = all_sentences_words + words_list
#             unq_words = set(words_list)
#             self.unique_words.update(unq_words)
#         word_counts = {i: all_sentences_words.count(i) for i in self.unique_words}
#         return self.unique_words, word_counts

#     def tokenize_words(self):
#         count_lst = [i for i in range(len(self.unique_words))]
#         word_idx = zip(self.unique_words, count_lst)
#         self.word_index = {w:i for w,i in word_idx}
#         idx_word = zip(count_lst, self.unique_words)
#         self.index_words = {i:w for i,w in idx_word}
#         return self.word_index, self.index_words






class CustomTokenizer:
    def __init__(self, sentences: List[str]):
        self.sentences = sentences
        self.index_words = dict()
        self.word_index = dict()
        self.unique_words = set()
        self._word_counts = None

    @property
    def unique_words_and_counts(self) -> Tuple[set, Dict[str, int]]:
        if self._word_counts is None:
            all_sentences_words = []
            for sentence in self.sentences:
                # sentence = '<start> ' + sentence
                words_list = sentence.split()
                all_sentences_words.extend(words_list)
                self.unique_words.update(words_list)

            self._word_counts = {word: all_sentences_words.count(word) for word in self.unique_words}
        return self.unique_words, self._word_counts

    @property
    def tokenized_words(self) -> Tuple[Dict[str, int], Dict[int, str]]:
        if not self.word_index or not self.index_words:
            count_lst = list(range(len(self.unique_words)))
            self.word_index = dict(zip(self.unique_words, count_lst))
            self.index_words = dict(zip(count_lst, self.unique_words))
        return self.word_index, self.index_words
    
    # @property
    def texts_to_sequences(self, list_of_sentences : List[List[str]]):
        final_list = []
        try:
            for sentence in list_of_sentences:
                sentence = "<start>" + " " + sentence
                word_indx_list = [self.word_index.get(word) for word in sentence.split() if word in self.word_index]
                final_list.append(word_indx_list)
        except Exception:
            raise ValueError('Error occured')
        return final_list


In [9]:
tokenizer = CustomTokenizer(text_data)

In [10]:
# unique_words, word_counts = tokenizer.find_unique_words()
unique_words, word_counts = tokenizer.unique_words_and_counts
print(word_counts)
print(unique_words)

{'hypothyroid': 9, 'problem': 1645, 'shaped': 1, 'nil': 1, 'approx.': 1, 'yrsd': 1, 'abdomen.': 7, 'operatively.': 1, 'month': 236, 'satisfactory': 4, 'nocturnal': 1, 'early': 1, '28': 17, 'cysts': 1, 'tsh': 2, 'cough.': 1, 'inflammatory': 1, '3months': 1, 'prof.': 1, 'again.': 1, 'ac.': 1, 'amp': 2, 'disappear': 1, 'abdomen2': 1, 'pressure': 1, 'similar': 2, 'open': 8, '20': 61, 'echoreflective': 1, 'neck': 7, 'dr.': 3, 'light': 658, 'gastric': 1, 'applied.': 1, 'good,': 1, 'rl': 1, 'change.usg': 1, 'same': 1, 'veg': 130, 'dm2': 2, 'loaded': 1, 'atrial': 1, 'gastritis': 2, '48': 1, 'healing': 2, 'consolidations.': 1, 'nature.': 2, 'now': 1, 'abdomen,': 2, 'umbilical': 21, 'hospital': 7, 'drained': 1, 'spicy': 13, 'present.': 8, 'put': 1, '2019': 1, 'improved': 2, 'largest': 1, 'stopped': 1, 'ischaemic': 1, 'rif': 4, 'paravertehal': 1, 'metron': 1, 'hypertensive.he': 1, 'defined': 1, 'shifted': 2, 'calculi,': 3, 'inj.': 5, 'decrease': 1, 'iv': 1, 'distended': 1, 'recorrent': 1, 'duch':

In [11]:
# word_idx, index_word = tokenizer.tokenize_words()
word_idx, index_word = tokenizer.tokenized_words
print(word_idx)
print(index_word)

{'hypothyroid': 0, 'problem': 1, 'shaped': 2, 'nil': 3, 'approx.': 4, 'yrsd': 5, 'abdomen.': 6, 'operatively.': 7, 'month': 8, 'satisfactory': 9, 'nocturnal': 10, 'early': 11, '28': 12, 'cysts': 13, 'tsh': 14, 'cough.': 15, 'inflammatory': 16, '3months': 17, 'prof.': 18, 'again.': 19, 'ac.': 20, 'amp': 21, 'disappear': 22, 'abdomen2': 23, 'pressure': 24, 'similar': 25, 'open': 26, '20': 27, 'echoreflective': 28, 'neck': 29, 'dr.': 30, 'light': 31, 'gastric': 32, 'applied.': 33, 'good,': 34, 'rl': 35, 'change.usg': 36, 'same': 37, 'veg': 38, 'dm2': 39, 'loaded': 40, 'atrial': 41, 'gastritis': 42, '48': 43, 'healing': 44, 'consolidations.': 45, 'nature.': 46, 'now': 47, 'abdomen,': 48, 'umbilical': 49, 'hospital': 50, 'drained': 51, 'spicy': 52, 'present.': 53, 'put': 54, '2019': 55, 'improved': 56, 'largest': 57, 'stopped': 58, 'ischaemic': 59, 'rif': 60, 'paravertehal': 61, 'metron': 62, 'hypertensive.he': 63, 'defined': 64, 'shifted': 65, 'calculi,': 66, 'inj.': 67, 'decrease': 68, 'i

In [12]:
print(len(unique_words))

1421


In [13]:
duplicate_index_word = index_word.copy()

tokenizer.word_index['<start>'] = 0
tokenizer.index_words[0] = '<start>'


tokenizer.word_index[duplicate_index_word.get(0)] = len(duplicate_index_word)
tokenizer.index_words[len(duplicate_index_word)] = duplicate_index_word.get(0)

In [14]:
word_idx = tokenizer.word_index
index_word = tokenizer.index_words

In [15]:
sequence = tokenizer.texts_to_sequences(text_data)
print(sequence)

[[0, 1045, 1320, 729, 105, 295, 1402, 426, 773, 8, 990, 105, 1, 990, 537, 495, 773, 8, 974, 990, 1191, 313, 340, 990, 1320, 1281, 990, 1198, 122, 997, 670, 871, 365, 990, 31, 967, 990, 122, 997, 886, 1411, 990, 1000, 390, 536, 990, 1260, 390, 995, 990, 106, 560, 990, 1168, 390, 597, 990, 85, 1213, 1137, 990, 1138, 1137, 990, 336, 691, 27, 289, 287, 990], [0, 708, 247, 947, 874, 186, 1031, 831, 338, 974, 990, 105, 1, 990, 537, 495, 831, 338, 974, 990, 785, 1, 990, 448, 1281, 990, 1198, 122, 997, 670, 871, 365, 990, 238, 967, 990, 467, 1411, 990, 910, 194, 990, 664, 442, 831, 618, 178, 903, 559, 990, 664, 694, 409, 694, 990, 1000, 390, 536, 990, 1260, 390, 995, 990, 638, 560, 990, 1168, 390, 597, 990, 85, 1213, 1137, 990, 1138, 1137, 990, 336, 691, 27, 289, 287, 990], [0, 1045, 629, 436, 1107, 426, 773, 456, 707, 785, 729, 915, 759, 436, 298, 990, 105, 1, 990, 537, 495, 773, 8, 974, 990, 1191, 313, 340, 990, 448, 1281, 990, 1198, 122, 997, 670, 871, 365, 990, 31, 967, 990, 122, 997, 886,

In [16]:
max_sequence_length = max_words_in_sentense_paragraph + 1

def pad_sequences(sequence_lists : List[List[int]], max_len: int, padding: Literal["pre","post"]):
    try:
        # blank_zero = np.zeros(shape= (len(sequence_lists), max_len), dtype='int32')
        blank_zero = torch.zeros([len(sequence_lists), max_len], dtype=torch.long).to(device)
        if padding == "pre":
            for i,sequence in enumerate(sequence_lists):
                for j in range(1, len(sequence)+1):
                    blank_zero[i][-j] = sequence[-j]
        else:
            for i,sequence in enumerate(sequence_lists):
                for j in range(len(sequence)):
                    blank_zero[i][j] = sequence[j]
        
        return blank_zero
    except Exception as e:
        raise ValueError("Error", e)

# def pad_sequences(sequence_lists: List[List[int]], max_len: int, padding: Literal["pre", "post"]) -> np.ndarray:
#     try:
#         # Create an array of zeros with shape (number of sequences, max_len)
#         blank_zero = np.zeros((len(sequence_lists), max_len), dtype='int32')

#         # Iterate over each sequence and its index
#         for i, sequence in enumerate(sequence_lists):
#             # Handle padding
#             if padding == "pre":
#                 # If padding is 'pre', place the sequence at the end of the array
#                 blank_zero[i, -len(sequence):] = sequence
#             elif padding == "post":
#                 # If padding is 'post', place the sequence at the beginning of the array
#                 blank_zero[i, :len(sequence)] = sequence
#             else:
#                 raise ValueError("Padding type must be either 'pre' or 'post'")

#         return blank_zero
#     except Exception as e:
#         raise ValueError(f"Error: {e}")


padded_sequences_post = pad_sequences(sequence, max_len=max_sequence_length, padding='post')
padded_sequences_pre = pad_sequences(sequence, max_len=max_sequence_length, padding='pre')

In [17]:
print(padded_sequences_post.shape)
print(padded_sequences_pre.shape)

torch.Size([1000, 349])
torch.Size([1000, 349])


In [18]:
# tokenizer_target = CustomTokenizer(target_data.values.flatten())      ##  for 21 columns

all_target_sentences_list = [' => '.join(list(value)) for value in target_data.values]
tokenizer_target = CustomTokenizer(' => '.join(list(value)) for value in target_data.values)

unique_words_target, word_counts_target = tokenizer_target.unique_words_and_counts
print(word_counts_target)
print(unique_words_target)

{'hypothyroid': 9, 'problem': 1645, 'shaped': 1, 'nil': 1, 'approx.': 1, 'yrsd': 1, 'abdomen.': 7, 'operatively.': 1, 'month': 236, 'satisfactory': 4, 'nocturnal': 1, 'early': 1, '28': 17, 'cysts': 1, 'tsh': 2, 'cough.': 1, 'inflammatory': 1, '3months': 1, 'prof.': 1, 'again.': 1, 'ac.': 1, 'amp': 2, 'disappear': 1, 'abdomen2': 1, 'pressure': 1, 'similar': 2, 'open': 8, '20': 61, 'echoreflective': 1, 'neck': 7, 'dr.': 3, 'light': 658, 'gastric': 1, 'applied.': 1, 'good,': 1, 'rl': 1, 'change.usg': 1, 'same': 1, 'veg': 130, 'dm2': 2, 'loaded': 1, 'atrial': 1, 'gastritis': 2, '48': 1, 'healing': 2, 'consolidations.': 1, 'nature.': 2, 'now': 1, 'abdomen,': 2, 'umbilical': 21, 'hospital': 7, 'drained': 1, 'spicy': 13, 'present.': 8, 'put': 1, '2019': 1, 'improved': 2, 'largest': 1, 'stopped': 1, 'ischaemic': 1, 'rif': 4, 'paravertehal': 1, 'metron': 1, 'hypertensive.he': 1, 'defined': 1, 'shifted': 2, 'calculi,': 3, 'inj.': 5, 'decrease': 1, 'iv': 1, 'distended': 1, 'recorrent': 1, 'duch':

In [19]:
max_words_in_sentense_target = max(len(i.split()) for i in all_target_sentences_list)
max_words_in_sentense_target = max_words_in_sentense_target + 1     ### 1 because of this( '<start>' )
print("max_words_in_sentense => ", max_words_in_sentense_target)

max_words_in_sentense =>  358


In [20]:
word_idx_target, index_word_target = tokenizer_target.tokenized_words
print(word_idx)
print(index_word_target)

{'hypothyroid': 1421, 'problem': 1, 'shaped': 2, 'nil': 3, 'approx.': 4, 'yrsd': 5, 'abdomen.': 6, 'operatively.': 7, 'month': 8, 'satisfactory': 9, 'nocturnal': 10, 'early': 11, '28': 12, 'cysts': 13, 'tsh': 14, 'cough.': 15, 'inflammatory': 16, '3months': 17, 'prof.': 18, 'again.': 19, 'ac.': 20, 'amp': 21, 'disappear': 22, 'abdomen2': 23, 'pressure': 24, 'similar': 25, 'open': 26, '20': 27, 'echoreflective': 28, 'neck': 29, 'dr.': 30, 'light': 31, 'gastric': 32, 'applied.': 33, 'good,': 34, 'rl': 35, 'change.usg': 36, 'same': 37, 'veg': 38, 'dm2': 39, 'loaded': 40, 'atrial': 41, 'gastritis': 42, '48': 43, 'healing': 44, 'consolidations.': 45, 'nature.': 46, 'now': 47, 'abdomen,': 48, 'umbilical': 49, 'hospital': 50, 'drained': 51, 'spicy': 52, 'present.': 53, 'put': 54, '2019': 55, 'improved': 56, 'largest': 57, 'stopped': 58, 'ischaemic': 59, 'rif': 60, 'paravertehal': 61, 'metron': 62, 'hypertensive.he': 63, 'defined': 64, 'shifted': 65, 'calculi,': 66, 'inj.': 67, 'decrease': 68,

In [21]:
duplicate_index_word_target = index_word_target.copy()

tokenizer_target.word_index['<start>'] = 0
tokenizer_target.index_words[0] = '<start>'


tokenizer_target.word_index[duplicate_index_word_target.get(0)] = len(duplicate_index_word_target)
tokenizer_target.index_words[len(duplicate_index_word_target)] = duplicate_index_word_target.get(0)

word_idx_target = tokenizer_target.word_index
index_word_target = tokenizer_target.index_words

In [22]:
            ## for 21 columns 
# target_sequences = np.zeros((len(df), target_data.shape[1], max_words_in_sentense_target), dtype='int32')
# for i, (key, sentences) in enumerate(target_data.items()):
#     sequences_target = tokenizer_target.texts_to_sequences(sentences)
#     padded_sequences_target = pad_sequences(sequences_target, max_len=max_words_in_sentense_target, padding='post')
#     target_sequences[:, i, :] = padded_sequences_target

In [23]:
sequences_target = tokenizer_target.texts_to_sequences(' => '.join(list(value)) for value in target_data.values)
padded_sequences_target = pad_sequences(sequences_target, max_len=max_words_in_sentense_target, padding='post')

In [24]:
padded_sequences_target.shape

torch.Size([1000, 358])

In [25]:
import torch
import torch.nn as nn


class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        self.values = nn.Linear(embed_size, embed_size)
        self.keys = nn.Linear(embed_size, embed_size)
        self.queries = nn.Linear(embed_size, embed_size)
        self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self, values, keys, query, mask):
        N = query.shape[0]

        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]
        # print(values.shape, keys.shape, query.shape)
        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(query)
        # print("====")
        # print(values.shape, keys.shape, queries.shape)
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = queries.reshape(N, query_len, self.heads, self.head_dim)
        # print(" ......... ")
        # print(values.shape, keys.shape, queries.shape)
        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        # print(energy.shape,  "   1     ")
        if mask is not None:

            # energy = energy.masked_fill(mask == 0, float("-inf"))     ## tensor error
            energy = energy.masked_fill(mask == 0, float("1e-10"))
        # print(energy.shape,  "    2    ")
        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)
        # print(attention.shape,  "   3    ")
        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )
        out = self.fc_out(out)
        return out

class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)

        self.norm1 = nn.LayerNorm(normalized_shape = embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)

        out = self.dropout(self.norm2(forward + x))
        return out


class Encoder(nn.Module):
    def __init__(self, src_vocab_size, embed_size, num_layers, heads, device,
                forward_expansion, dropout, max_length,):
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)

        self.position_embedding = nn.Embedding(max_length, embed_size)
        self.layers = nn.ModuleList(
            [
                TransformerBlock(embed_size, heads, dropout=dropout,forward_expansion=forward_expansion) for _ in range(num_layers)
            ]
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)

        out = self.dropout(
            (self.word_embedding(x) + self.position_embedding(positions))
        )
        for layer in self.layers:
            out = layer(out, out, out, mask)
        return out


class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        self.norm = nn.LayerNorm(embed_size)
        self.attention = SelfAttention(embed_size, heads=heads)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):
        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)
        return out


class Decoder(nn.Module):
    def __init__( self, trg_vocab_size, embed_size, num_layers, heads, forward_expansion, dropout, device, max_length):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)
        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
                for _ in range(num_layers)
            ]
        )
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        x = self.dropout((self.word_embedding(x) + self.position_embedding(positions)))
        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)

        out = self.fc_out(x)
        return out


class Transformer(nn.Module):
    def __init__( self, src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, embed_size=128, # embed_size=512, 
                 num_layers=6, forward_expansion=4, heads=8, dropout=0, device="gpu", max_length=500,):
        super(Transformer, self).__init__()
        self.encoder = Encoder(src_vocab_size, embed_size, num_layers, heads, 
                               device, forward_expansion, dropout, max_length,)

        self.decoder = Decoder(trg_vocab_size, embed_size, num_layers, heads, forward_expansion, dropout, device, max_length)

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        return src_mask.to(self.device)

    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(N, 1, trg_len, trg_len)
        return trg_mask.to(self.device)


    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out



In [26]:
from torch.utils.data import DataLoader, TensorDataset


x = padded_sequences_post
trg = padded_sequences_target


dataset = TensorDataset(x, trg)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [27]:
import torch.optim as optim


# src_vocab_size = max_words_in_sentense_paragraph + 1
# trg_vocab_size = max_words_in_sentense_target
src_vocab_size = len(index_word)
trg_vocab_size = len(index_word_target)
src_pad_idx = 0
trg_pad_idx = 0

# Initialize the model
model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, device=device).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=model.trg_pad_idx)
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [28]:
def train_model(model, dataloader, criterion, optimizer, device):
    model.train()
    num_epochs = 10
    for epoch in range(num_epochs):
        total_loss = 0
        for src, trg in dataloader:
            src = src.to(device)
            trg = trg.to(device)
            try:
                # Forward pass
                output = model(src, trg[:, :-1])
                # Calculate loss
                trg_y = trg[:, 1:].contiguous().view(-1)
                output = output.view(-1, output.size(-1))
                loss = criterion(output, trg_y)
                # Backward pass and optimization
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            except RuntimeError as e:
                print(f"Error during training: {e}")
                torch.cuda.synchronize()  # Ensure all CUDA operations are complete
                raise  # Re-raise the error for further inspection
        avg_loss = total_loss / len(dataloader)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')

# Train the model
train_model(model, dataloader, criterion, optimizer, device)

Epoch 1/10, Loss: 1.4424
Epoch 2/10, Loss: 0.7514
Epoch 3/10, Loss: 0.6500
Epoch 4/10, Loss: 0.5720
Epoch 5/10, Loss: 0.5033
Epoch 6/10, Loss: 0.4350
Epoch 7/10, Loss: 0.3950
Epoch 8/10, Loss: 0.3550
Epoch 9/10, Loss: 0.3207
Epoch 10/10, Loss: 0.2915


In [29]:
def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    
    total_loss = 0
    with torch.no_grad():
        for src, trg in dataloader:
            src = src.to(device)
            trg = trg.to(device)
            
            output = model(src, trg[:, :-1])
            
            trg_y = trg[:, 1:].contiguous().view(-1)
            output = output.view(-1, output.size(-1))
            loss = criterion(output, trg_y)
            
            total_loss += loss.item()
    
    avg_loss = total_loss / len(dataloader)
    print(f'Validation Loss: {avg_loss:.4f}')

# Evaluate the model
evaluate_model(model, dataloader, criterion, device)


Validation Loss: 0.2438


In [30]:
# # Save the model
# torch.save(model.state_dict(), 'transformer_model.pth')

# # Load the model
# model = Transformer(
#     src_vocab_size=10000,
#     trg_vocab_size=10000,
#     src_pad_idx=0,
#     trg_pad_idx=0,
#     embed_size=512,
#     num_layers=6,
#     forward_expansion=4,
#     heads=8,
#     dropout=0.1,
#     device=device,
#     max_length=100
# ).to(device)
# model.load_state_dict(torch.load('transformer_model.pth'))


In [31]:
# def preprocess_sentence(sentence, vocab, max_length):
#     tokens = tokenize_text(sentence.lower())
#     token_ids = tokens_to_ids(tokens, vocab)
#     padded_ids = pad_sequences([token_ids], max_len=max_length).squeeze(0)
#     return torch.tensor(padded_ids).unsqueeze(0)  # Add batch dimension

# output_tokenizer = OutputCustomTokenizer(text_data)

def preprocess_sentence(sentence, vocab, max_length):
    tokens = tokenizer.texts_to_sequences([sentence])
    padded_ids = pad_sequences(tokens, max_len=max_length, padding='post').squeeze(0)       ## max_sequence_length
    return torch.tensor(padded_ids).unsqueeze(0)  # Add batch dimension


In [32]:
# def encode_input(model, input_tensor, device):
#     model.eval()
#     input_tensor = input_tensor.to(device)
#     src_mask = create_padding_mask(input_tensor)
#     with torch.no_grad():
#         encoder_output = model.encoder(input_tensor, src_mask)
#     return encoder_output


def encode_input(model, input_tensor, device):
    model.eval()
    input_tensor = input_tensor.to(device)
    src_mask = model.make_src_mask(input_tensor)
    with torch.no_grad():
        encoder_output = model.encoder(input_tensor, src_mask)
    return encoder_output

In [33]:
for zzz, qqq in dataloader:
    src_zzz = zzz.to(device)
    trg_qqq = qqq.to(device)
    print(src_zzz.shape)
    print(trg_qqq.shape)
    break

torch.Size([2, 349])
torch.Size([2, 358])


In [34]:
# torch.tensor([1,2,3,4,5,6,7,8,9,0]).reshape([1,10])

In [35]:
def decode_sequence(model, encoder_output, device, trg_vocab, max_length, start_token='<start>', end_token = '<start>'): ## end_token='<eos>'):
    model.eval()
    
    # Initialize the target sequence with the start token
    trg_tensor = torch.tensor([trg_vocab[start_token]]).unsqueeze(0).to(device)  # Add batch dimension
    trg_seq = [trg_vocab[start_token]]
    
    for _ in range(max_length):
        # src_mask = model.make_src_mask(encoder_output).squeeze(0).to(device)
        # trg_mask = model.make_trg_mask(trg_tensor).to(device)

        def make_src_mask(src):
            src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
            return src_mask.to(device)

        def make_trg_mask(trg):
            N, trg_len = trg.shape
            trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(N, 1, trg_len, trg_len)
            return trg_mask.to(device)

        src_mask = make_src_mask(encoder_output[:, :, -1])
        trg_mask = make_trg_mask(trg_tensor)
        
        with torch.no_grad():           ### x, enc_out, src_mask, trg_mask
            output = model.decoder(trg_tensor, encoder_output, src_mask, trg_mask)
        
        # Get the most probable next token
        next_token = output.argmax(dim=-1)[:, -1].item()
        trg_seq.append(next_token)
        
        if next_token == trg_vocab[end_token]:
            break
        
        # Update target tensor for the next step
        trg_tensor = torch.cat((trg_tensor, torch.tensor([[next_token]]).to(device)), dim=1)
    
    return trg_seq


In [36]:
def ids_to_tokens(ids, vocab):
    inv_vocab = {v: k for k, v in vocab.items()}
    return [inv_vocab.get(id, '<unk>') for id in ids]

# # Example usage
# predicted_ids = decode_sequence(model, encoder_output, device, target_vocab, max_length=50)
# predicted_sentence = ' '.join(ids_to_tokens(predicted_ids, target_vocab))
# print(predicted_sentence)


In [37]:
# Define vocabularies and constants
src_vocab = len(tokenizer.word_index)  # Source vocabulary
trg_vocab = word_idx_target  # Target vocabulary
max_length = max_words_in_sentense_target  # Max length of sequences

# Preprocess input sentence
input_sentence = "Ho intermittent pain in the RIFX 7 days. It was associated with vomiting . Abdomen problem . Complains from 1 week ago . Suddenly it happen . Intermittent attack . Patient does not have smoking habit . Light Worker . Does not drink alcohol . Non Vegetarian . Prefer oil mustrad oil . Treatment type OPERATIVE . Surgery type APPENDICECTOMY . 50 minutes . Anesthic type General . Anes smooth recovery . NORMAL recovery . Follow-up date 27 December 2014 ."
input_tensor = preprocess_sentence(input_sentence, src_vocab, max_length)

# print(input_tensor.shape)

# Encode the input
encoder_output = encode_input(model, input_tensor, device)

# Decode the output sequence
predicted_ids = decode_sequence(model, encoder_output, device, trg_vocab, max_length)
predicted_sentence = ' '.join(ids_to_tokens(predicted_ids, trg_vocab))

print(predicted_sentence)

C:\Users\jites\AppData\Local\Temp\ipykernel_21184\1754940933.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(padded_ids).unsqueeze(0)  # Add batch dimension


<start> breast detected during health check up => abdomen problem => nan => no attack => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan not have february 2015 usg april 2015 => nan => nan => nan => nan => nan => nan => nan => nan april 2015 attack => nan 2015 => nan => nan => nan april 2015 a day hypertensive usg detect 15 july 2013 => nan => nan => nan march 20 february april 2015 a day with biopsy report after 7 days with biopsy report after 7 days with biopsy report after 7 days with to to all over february 2015 => nan => nan health attack => nan => nan for july 2015 => nan => nan => nan => nan => nan => nan february 2015 => nan => nan april 2015 => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan => nan was july 2015 hypertensive multiple no attack => nan => nan => nan => 

In [38]:
# if torch.isnan(x).any() or torch.isinf(x).any():
#     raise ValueError("Tensor contains NaNs or Infs")


In [39]:
max_index = x.max().item()  # Get the maximum index used in src tensor
assert max_index < src_vocab_size, f"Index out of range: {max_index} >= {src_vocab_size}"

In [40]:
embedding = nn.Embedding(num_embeddings=src_vocab_size, embedding_dim=512)
print("Embedding matrix shape:", embedding.weight.shape)


Embedding matrix shape: torch.Size([1422, 512])


In [41]:
assert src_pad_idx < src_vocab_size, f"Padding index out of range: {src_pad_idx} >= {src_vocab_size}"